# AEMO (Max demand daily) + BOM weather data

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import os

In [2]:
#loading our electricity demand data
df = pd.read_csv("../Data/edited/AEMO_merged/combined_csv-operational-demand.csv")
df = df.sort_values('timestamp')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.head()

,timestamp,Operational Demand (MW)
0,2006-09-20 00:00:00,1347.418
1,2006-09-20 00:30:00,1312.708
2,2006-09-20 01:00:00,1298.644
3,2006-09-20 01:30:00,1288.972
4,2006-09-20 02:00:00,1282.006


In [8]:
df_bom= pd.read_csv("../Data/edited/HistoricalWeather/perth_airport.csv",index_col=[0])
df_bom = df_bom.drop(columns = ['Station','Transpiration','Rain','Evaporation'])
df_bom['Date'] = pd.to_datetime(df_bom['Date'])
df_bom.head()

,Date,Max Temp,Min Temp,Max Hum,Min Hum,Av Wind,Solar Rad
0,2009-01-01,37.2,18.8,65,20,6.1,34.01
1,2009-01-02,36.1,21.9,70,25,5.31,34.25
2,2009-01-03,34.4,20.3,79,29,5.45,34.11
3,2009-01-04,36.5,18.5,62,20,5.25,34.05
4,2009-01-05,37.2,21.8,63,17,6.77,34.18


In [4]:
dataset = df
dataset["Date"] = pd.to_datetime(df['timestamp']).dt.date
#dataset["Time"] = pd.to_datetime(df["Datetime"], errors='coerce').dt.time
dataset.index = pd.to_datetime(dataset.index)
dataset['Date']= pd.to_datetime(dataset['Date'])
dataset.info()


/var/folders/x3/zqbjwpy10g30gjb8chvrw95h0000gn/T/ipykernel_54400/1427902258.py:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  dataset["Week"] = pd.to_datetime(df['timestamp']).dt.week


In [6]:
#We are aggregating demand by their max demand on the daily basis (based on 30 mins interval)
agg_aemo = pd.DataFrame(dataset.groupby(['Date'])['Operational Demand (MW)'].max().reset_index())
agg_aemo.head()

,Date,Operational Demand (MW)
0,2006-09-20,2168.548
1,2006-09-21,2272.658
2,2006-09-22,2126.978
3,2006-09-23,1969.044
4,2006-09-24,1933.980


In [11]:
df_merged = pd.merge(agg_aemo, df_bom, on='Date', how='inner')
df_merged = df_merged.rename(columns ={'Operational Demand (MW)' : 'Max Operation Demand (MW)'})
df_merged.head()


,Date,Max Operation Demand (MW),Max Temp,Min Temp,Max Hum,Min Hum,Av Wind,Solar Rad
0,2009-01-01,2543.140,37.2,18.8,65,20,6.1,34.01
1,2009-01-02,2893.868,36.1,21.9,70,25,5.31,34.25
2,2009-01-03,2450.920,34.4,20.3,79,29,5.45,34.11
3,2009-01-04,2526.302,36.5,18.5,62,20,5.25,34.05
4,2009-01-05,3106.014,37.2,21.8,63,17,6.77,34.18


In [12]:
#save the csv to folder
df_merged.to_csv("../Data/edited/aemo_max_daily_bom_merged.csv")